# INSTALL & IMPORTS

In [1]:
#pip install selenium -q
#pip install requests -q
# pip install bs4 -q 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# TESTING

In [42]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [81]:
page_source = driver.page_source
profile = BeautifulSoup(page_source, 'html.parser')
name = profile.find("span", class_='encounters-story-profile__name').text
age = profile.find("span", class_= "encounters-story-profile__age").text
about = profile.find("ul", class_= "encounters-story-about__badges").text if profile.find("ul", class_= "encounters-story-about__badges") else ""
if about:
    about = re.split('(?=[A-Z])', about)

location = profile.find("span", class_="header-2 text-color-black").text
more_info = profile.find("p", class_="encounters-story-about__text").text
job = profile.find("p", class_="encounters-story-profile__occupation").text
education = profile.find("p", class_="encounters-story-profile__education").text


print(name, age, about, location, more_info, job, education)

Joan , 30 ['174 cm', 'Active', 'Aries', 'Graduate degree', 'Socially', 'Never', 'Man', 'Something casual', 'Open to kids', 'Agnostic'] Barcelona Buscant còmplice per tast de cerveses

That's what she said Advocat Universitat Pompeu Fabra 2015


In [83]:
education

'Universitat Pompeu Fabra 2015'

In [54]:
# Esperar a que el botón esté presente y sea clickeable
try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
    )
    button.click()
    print("Botón clickeado con éxito")
except Exception as e:
    print("No se pudo clickear el botón:", e)


Botón clickeado con éxito


# FUNCTION TO TAKE ALL INFO

In [13]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [33]:
def extract_profile_info():

    profiles = []
    prevJob = ""
    prevStories = ""
    prevEducation = ""
    
    for i in range(200):
        
        # Obtener el contenido de la página actual
        page_source = driver.page_source
    
        # Analizar el contenido de la página con BeautifulSoup
        profile = BeautifulSoup(page_source, 'html.parser')
    
    
        # Realizar un diccionario para que la información se vaya guardando.
        profiles_dict = {}
    
        # Esperar 15 segundos antes de extraer la información
        time.sleep(5)
    
        # Extraer información del perfil
        name = profile.find("span", class_='encounters-story-profile__name')
        profiles_dict["name"] = name.text if name else None
        
        age = profile.find("span", class_= "encounters-story-profile__age")
        profiles_dict["age"] = age.text.strip(",")  if age else None

        
        location = profile.find("span", class_="header-2 text-color-black")
        profiles_dict["location"] = location.text if location else ""
    

        story_about = profile.find("p", class_="encounters-story-about__text")
        profiles_dict["story_about"] = story_about.text if story_about and story_about.text != prevStories else None
        if story_about:
            prevStories = story_about.text

        
        job = profile.find("p", class_="encounters-story-profile__occupation")
        profiles_dict["job"] = job.text if job and job.text != prevJob else None
        if job:
            prevJob = job.text
    
        education = profile.find("p", class_="encounters-story-profile__education")
        profiles_dict["education"] = education.text if education and education.text != prevEducation else None
        if education:
            prevEducation = education.text

        # Encuentra todos los elementos div con la clase "pill"
        about = profile.find_all("div", class_="pill")  

        #categories_dict = {}
        
        for pill in about: # Pill como elemento de la lista
            image_tag = pill.find("img")
            
            if image_tag is not None: # Verificación de la existencia de una etiqueta de imagen
                image_url = image_tag['src'] # Extracción de la URL de la imagen
                text = pill.get_text(strip=True) # Extracción de texto del elemento pill 
                # strip=True se utiliza para eliminar espacios en blanco alrededor del texto
                category = image_url.split('_')[-1].split('.')[0] # Extracción de la categoría de la imagen
                if category is not None and category not in profiles_dict: 
                    # Verificación de la existencia de la categoría y actualización del diccionario
                    # Asignación de valores None cuando no se encuentra una etiqueta de imagen
                    profiles_dict[category] = text # Add to profiles
            
            else:
                category = None
                text = None
                
    

        profiles.append(profiles_dict)
    
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
            )
            time.sleep(5)  # Añadir un temporizador adicional de 5 segundos
            button.click()
            print("Botón clickeado con éxito")
        except Exception as e:
            print("No se pudo clickear el botón:", e)
            

    return profiles

In [34]:
profile_info = extract_profile_info() 

# Make a DataFrame from a dict. 
df = pd.DataFrame(profile_info)

# Print DataFrame
df

Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clickeado con éxito
Botón clicke

,name,age,location,story_about,job,education,heightv2,educationv2,drinkingv2,familyPlansv2,Politicsv2,religionv2,exercisev2,smokingv2,genderv2,intentionsv2,starSignv2
0,Annie,26,Barcelona,"bookish, into ambient music, painting, meditat...",None,None,173 cm,Graduate degree,Socially,Don’t want,Left,Agnostic,NaN,NaN,NaN,NaN,NaN
1,Elena,24,Barcelona,📍BCN\n\n🌍Travel ⛰️hiking 🎵music 🍜 cooking 🐥wor...,None,None,168 cm,In grad school,Socially,Don’t want,Left,Agnostic,Sometimes,Never,Woman,Relationship,Scorpio
2,André,25,Barcelona,hola :)\n\nun portugués en Barcelona \n\n@andr...,None,Nova SBE 2022,180 cm,In grad school,Socially,Open to kids,Left,NaN,Sometimes,Never,Man,Relationship,Virgo
3,Walid,25,"L'Hospitalet de Llobregat, Barcelona",None,None,None,180 cm,In grad school,Socially,Open to kids,NaN,NaN,Sometimes,NaN,Man,Relationship,Virgo
4,Artur,25,Barcelona,None,None,None,173 cm,Graduate degree,Socially,Not sure yet,NaN,Atheist,Active,Never,Man,Relationship,Leo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Maria,25,"Cornellà de Llobregat, Barcelona",None,None,None,164 cm,Graduate degree,Socially,Want someday,NaN,NaN,Active,Never,Woman,Don’t know yet,Leo
196,Euge,28,Barcelona,Busco a alguien con quien divertirme y sorpren...,Profesora De Baile at Farrays Dance Center,None,158 cm,Trade/tech school,Socially,Not sure yet,NaN,NaN,Active,Socially,Woman,Something casual,Sagittarius
197,Mireia,26,Barcelona,Ir en metro🧍🏼‍♀️,None,None,165 cm,Undergraduate degree,Socially,Not sure yet,NaN,NaN,Active,Socially,Woman,Relationship,Aquarius
198,Alexia,24,Barcelona,Latina de Costa Rica❤️\n📍 viviendo en Barcelona,Entrepreneur,None,165 cm,Undergraduate degree,Socially,NaN,NaN,NaN,Active,Never,Woman,Relationship,Sagittarius


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           200 non-null    object
 1   age            200 non-null    object
 2   location       200 non-null    object
 3   story_about    138 non-null    object
 4   job            62 non-null     object
 5   education      45 non-null     object
 6   heightv2       194 non-null    object
 7   educationv2    180 non-null    object
 8   drinkingv2     191 non-null    object
 9   familyPlansv2  147 non-null    object
 10  Politicsv2     90 non-null     object
 11  religionv2     115 non-null    object
 12  exercisev2     187 non-null    object
 13  smokingv2      179 non-null    object
 14  genderv2       190 non-null    object
 15  intentionsv2   183 non-null    object
 16  starSignv2     184 non-null    object
dtypes: object(17)
memory usage: 26.7+ KB


In [36]:
df.to_csv('man3.csv', index=False)